In [2]:
import pandas as pd
import os
import requests
from dotenv import load_dotenv

Variables List + Header API

In [9]:
load_dotenv()
RIOT_API_KEY = os.getenv("RIOT_API_KEY")
my_game_name = os.getenv("my_game_name")
my_tag_line = os.getenv("my_tag_line")

In [10]:
headers = {
    "X-Riot-Token" : RIOT_API_KEY
}

# Function to get puuid using summoner_id from Dataframe

In [11]:
df_only_summ_ids = pd.read_csv("all_wanted_summoner_ids.csv")
df_only_summ_ids = df_only_summ_ids.head(1)

for index, row in df_only_summ_ids.iterrows():
        summoner_id = row["summonerId"]
        api_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
        try:
            resp = requests.get(api_url, headers=headers)  #Insert API link here
            resp.raise_for_status()
            summoner_info = resp.json()
            df_only_summ_ids.iloc[index, df_only_summ_ids.columns.get_loc("puuid")] = summoner_info.get("puuid", "Not Found") # Store puuid in df, output Not Found if KeyError
        except requests.HTTPError:
            print(f"Couldn't complete request for {summoner_id}")


df_only_summ_ids

Couldn't complete request for rlesU8V-exQlYK_vWUX1KMYJBlSYRMS2FjmFxcQtsOcTtsBXJ4d9xIRbQA


,Unnamed: 0,summonerId
0,0,rlesU8V-exQlYK_vWUX1KMYJBlSYRMS2FjmFxcQtsOcTts...


In [ ]:
def get_puuid(csv_dataframe):
    # Assigning csv file name here as a placeholder
    csv_dataframe = "all_wanted_summoner_ids.csv"
    # Read CSV
    df_only_summ_ids = pd.read_csv(csv_dataframe)
    # Create an empty column for puuid
    df_only_summ_ids["puuid"] = ""
    # Loop through summonerId column
    for index, row in df_only_summ_ids.iterrows():
        summoner_id = row["summonerId"]
        api_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
        try:
            resp = requests.get(api_url, headers=headers)  #Insert API link here
            resp.raise_for_status()
            summoner_info = resp.json()
            df_only_summ_ids.iloc[index, df_only_summ_ids.columns.get_loc("puuid")] = summoner_info.get("puuid", "Not Found") # Store puuid in df, output Not Found if KeyError
        except requests.HTTPError:
            print(f"Couldn't complete request for {summoner_id}")
            # Save updated df back to CSV file
    df_only_summ_ids.to_csv("summoner_ids_with_puuids.csv", index=False)
    return df_only_summ_ids

# Function to get match ids using puuids from Dataframe

In [ ]:
def get_match_ids(csv_dataframe):
    # Assigning csv file name here as a placeholder
    csv_dataframe = "summoner_ids_with_puuids.csv"
    # Read CSV
    df_summid_and_puuid = pd.read_csv(csv_dataframe)
    # Create empty DataFram with puuid, summid and match_id columns
    df_puuids_and_match_ids = pd.DataFrame(columns = ["summonerId", "puuid", "match_ids"])
    # Loop through puuid column
    for index, puuid in df_summid_and_puuid["puuid"]:
        api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100"
        try:
            resp = requests.get(api_url, headers = headers)
            resp.raise_for_status()
            match_ids = resp.json()
            for 
            df_puuids_and_match_ids.at[index, "match_ids"] = match_ids.get("match_ids", "Not Found") # Store puuid in df, output Not Found if KeyError
        except requests.HTTPError:
            print(f"Couldn't complete request for {puuid}")
    # Save updated df back to CSV file
    df.to_csv(csv_dataframe, index=False)
    return df


# Function to get list of match_ids using puuid
Need to create a loop to keep providing the next 100 ids due to count limit, important to care with rate limit

In [ ]:
def get_match_id(puuid):
    api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100"
    try:
        resp = requests.get(api_url, headers=headers )  #Insert API link here
        resp.raise_for_status()
    except requests.HTTPError:
        print("Couldn't complete request")
    match_ids = resp.json()
    return match_ids